<a href="https://colab.research.google.com/github/jatoogunhyo/sec_edgar/blob/main/US_listed_comp_raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요한 환경 설정

In [1]:
import pandas as pd
import time
from tqdm.notebook import tqdm  # 구글 코랩에 적합한 tqdm 모듈
import requests
import warnings

# SettingWithCopyWarning 비활성화
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

# 미국 상장사 목록 불러오기

## ①SEC 사이트에서 불러오기
* CIK를 확보하기 위함

In [2]:
# SEC에서 제공하는 CIK 목록
url_ciks = 'https://www.sec.gov/files/company_tickers_exchange.json'
# headers는 SEC에서 이메일 입력을 요구함
headers = {'User-Agent' : 'JaeKwang (jaekwang.jung@krx.co.kr)'}
# 불러오기
res = requests.get(url_ciks, headers=headers)
cik_list = res.json()
cik_df = pd.DataFrame(cik_list['data'], columns=cik_list['fields'])

## ②Nasdaq 사이트 Screener에서 불러오기
* NYSE, Nasdaq이 함께 있음
* 장점 : 시가총액, 보통주 구분이 함께 기재되어 있음
* 단점 : 시장구분자, 보통주구분자가 없음

In [3]:
# Nasdaq screener URL
url_nasdaq = 'https://api.nasdaq.com/api/screener/stocks?tableonly=false&limit=10000'

# 요청 헤더
headers =  {'User-Agent': 'Mozilla/5.0'}

# GET 요청 보내기
res = requests.get(url_nasdaq, headers=headers)

# JSON으로 받은 내용 데이터프레임으로 전환
df_nasdaq = pd.DataFrame(res.json()['data']['table']['rows'])

In [4]:
# 보통주의 class와 관련하여 SEC의 법인 목록에는 '-', Nasdaq 법인 목록에는 '/'가 사용됨
# ex. 버크셔헤서웨이 (SEC) BRK-A / (Nasdaq) BRK.A
# 따라서 Nasdaq SEC에 맞춰줌
df_nasdaq['symbol'] = df_nasdaq.apply(lambda x: x['symbol'].replace('/', '-'), axis=1)

# 시가총액이 없는 것은 제외함
df_nasdaq_mkt = df_nasdaq[df_nasdaq['marketCap'] != 'NA']

## ③NYSE 사이트에서 불러오기
* NYSE, Nasdaq이 함께 있음
* 장점 : 시장구분자, 보통주구분자가 함께 달려있음
* 단점 : 시가총액이 없음

📣 거래소 코드


| MIC 코드 | 거래소/시장                      | 설명                                                   |
|----------|----------------------------------|-------------------------------------------------------|
| XNYS     | New York Stock Exchange (NYSE)  | 전통적인 대형 거래소, 대형 다국적 기업 중심             |
| XNCM     | NASDAQ Capital Market           | 중소형 성장 기업 중심의 나스닥 세그먼트                |
| XNGS     | NASDAQ Global Select Market     | 나스닥에서 가장 엄격한 상장 기준을 충족하는 대형 기업 중심 |
| XNMS     | NASDAQ Global Market            | 기술 중심 기업 및 글로벌 중소형·대형 기업 포함          |
| XASE     | NYSE American (AMEX)            | 소형주 및 ETF 중심의 거래소, 이전 명칭은 AMEX          |
| BATS     | BATS Global Markets             | 전자 거래 중심의 글로벌 증권 거래소                    |


In [5]:
# NYSE API URL
url_nyse = 'https://www.nyse.com/api/quotes/filter'

# 요청 매개변수
params = {
    'filterToken': None,  # 필수값이 아니라면 None
    'instrumentType': 'EQUITY',
    'maxResultsPerPage': 10000,
    'pageNumber': 1,
    'sortColumn': 'NORMALIZED_TICKER',
    'sortOrder': 'ASC'
}

# 요청 헤더
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    'Content-Type': 'application/json'
}

# POST 요청 보내기
res = requests.post(url_nyse, json=params, headers=headers)

# 응답 확인
df_nyse = pd.DataFrame(res.json())
df_nyse.head()

,total,url,exchangeId,instrumentType,symbolTicker,symbolExchangeTicker,normalizedTicker,symbolEsignalTicker,instrumentName,micCode
0,6723,https://www.nyse.com/quote/XNYS:A,558,COMMON_STOCK,A,A,A,A,AGILENT TECHNOLOGIES INC,XNYS
1,6723,https://www.nyse.com/quote/XNYS:AA,558,COMMON_STOCK,AA,AA,AA,AA,ALCOA CORPORATION,XNYS
2,6723,https://www.nyse.com/quote/XNMS:AACG,564,DEPOSITORY_RECEIPT,AACG,AACG,AACG,AACG,ATA CREATIVITY GLOBAL SPON ADS EACH REP 2 ORD SHS,XNMS
3,6723,https://www.nyse.com/quote/XNYS:AACT,558,COMMON_STOCK,AACT,AACT,AACT,AACT,ARES ACQUISITION CORPORATION II,XNYS
4,6723,https://www.nyse.com/quote/XNYS:AACT.U,558,UNIT,AACT.UN,AACT.U,AACT.U,AACT.U,ARES ACQUISITION CORPORATION II UNIT 1 CL A & ...,XNYS


NYSE에서 받은 상장종목 목록의 Instrument Type은 다음이 있음:
* 'COMMON_STOCK'
* 'DEPOSITORY_RECEIPT'
* 'UNIT'
* 'REIT'
* 'LIMITED_PARTNERSHIP'
* 'PREFERRED_STOCK'
* 'CLOSED_END_FUND'
* EXCHANGE_TRADED_NOTE'
* 'TRUST'
* 'UNITS_OF_BENEFICIAL_INTEREST'

**⇒ 분석대상 종목은 보통주로 한정**

In [6]:
# NYSE법인에서 보통주로 한정
df_nyse_cs = df_nyse[df_nyse['instrumentType'] == 'COMMON_STOCK']

# 보통주의 class와 관련하여 SEC의 법인 목록에는 '-', NYSE 법인 목록에는 '.'가 사용됨
# ex. 버크셔헤서웨이 (SEC) BRK-A / (NYSE) BRK.A
# 따라서 NYSE것을 SEC에 맞춰줌
df_nyse_cs['symbolTicker'] = df_nyse_cs.apply(lambda x: x['symbolTicker'].replace('.', '-'), axis=1)

## ➡ ①, ②, ③을 결합
1) 각 데이터프레임별 칼럼 :
* cik_df_ny : ['cik', 'name', 'ticker', 'exchange']
* df_nasdaq_mkt : ['symbol', 'name', 'lastsale', 'netchange', 'pctchange', 'marketCap', 'url']
* df_nyse_cs : ['total', 'url', 'exchangeId', 'instrumentType', 'symbolTicker', 'symbolExchangeTicker', 'normalizedTicker', 'symbolEsignalTicker', 'instrumentName', 'micCode']

2) 결합 방향
* NYSE 목록이 보통주만 추린 것이므로,
* NYSE를 중심으로 SEC(CIK 취득)와 Nasdaq(시총 취득)을 붙임

In [7]:
# NYSE 보통주 목록에 Nasdaq 목록을 결합
df_listed = pd.merge(df_nyse_cs[['symbolTicker', 'instrumentName', 'micCode']], df_nasdaq[['symbol', 'marketCap']],
                     left_on='symbolTicker', right_on='symbol', how='left')

# NYSE+Nasdaq 보통주 목록에 SEC 목록을 결합
df_listed = pd.merge(df_listed, cik_df[['ticker', 'cik', 'exchange']],
                     left_on='symbolTicker', right_on='ticker', how='left')

# 결합 후 cik 열이 float으로 변환된 경우 int로 다시 변환
if df_listed['cik'].dtype == 'float64':
    df_listed['cik'] = df_listed['cik'].astype('Int64')  # Pandas의 nullable int 사용

# 시가총액이 없는 데이터를 제거
df_listed = df_listed[df_listed['marketCap'] != 'NA']

# micCode에 따라 exchange_name을 매핑
mic_to_exchange = {
    'XNYS': 'NYSE',
    'XNCM': 'Nasdaq CM',
    'XNGS': 'Nasdaq GS',
    'XNMS': 'Nasdaq GM',
    'XASE': 'AMEX',
    'BATS': 'BATS'
}
df_listed['exchange_segment'] = df_listed['micCode'].map(mic_to_exchange)

# Nasdaq에 해당하는 micCode를 그룹으로 통일
nasdaq_codes = ['XNCM', 'XNGS', 'XNMS']
df_listed['exchange_name'] = df_listed['micCode'].apply(lambda x: 'Nasdaq' if x in nasdaq_codes else mic_to_exchange.get(x, 'OTHER'))

# 칼럼정리
df_listed = df_listed[['symbolTicker', 'cik', 'instrumentName', 'exchange_name', 'exchange_segment','marketCap']]

# 칼럼명정리
df_listed.columns = ['symbol', 'cik', 'name', 'exchange', 'exchange_segment','marketCap']

# 천 단위 구분 콤마 제거 후 숫자형으로 변환
df_listed['marketCap'] = df_listed['marketCap'].str.replace(',', '').astype(float)

# marketCap이 큰 순서로 정렬
df_listed = df_listed.sort_values(by='marketCap', ascending=False)

# 확인된 상장종목 수
print(f"상장종목 수: {len(df_listed)}")

# marketCap 상위 100 개 기업만 필터
df_listed_k = df_listed.head(100)
df_listed_k

상장종목 수: 4688


,symbol,cik,name,exchange,exchange_segment,marketCap
11,AAPL,320193,APPLE INC,Nasdaq,Nasdaq GS,3.668610e+12
3095,NVDA,1045810,NVIDIA CORP,Nasdaq,Nasdaq GS,3.431294e+12
2873,MSFT,789019,MICROSOFT CORP,Nasdaq,Nasdaq GS,3.156553e+12
1889,GOOG,1652044,ALPHABET INC,Nasdaq,Nasdaq GS,2.391769e+12
1890,GOOGL,1652044,ALPHABET INC,Nasdaq,Nasdaq GS,2.374142e+12
...,...,...,...,...,...,...
69,ADI,6281,ANALOG DEVICES INC,Nasdaq,Nasdaq GS,1.094037e+11
4448,UPS,1090727,UNITED PARCEL SERVICE INC,NYSE,NYSE,1.081731e+11
794,CB,896159,CHUBB LIMITED,NYSE,NYSE,1.074861e+11
4575,VRTX,875320,VERTEX PHARMACEUTICAL,Nasdaq,Nasdaq GS,1.061304e+11
